---
# I. Installs & Imports

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~ Required Installs ~~~~~~~~~~~~~~~~~~~~~~~ #
!pip install torch --index-url https://download.pytorch.org/whl/cu121
!pip install torchvision
!pip install numpy pandas matplotlib seaborn scikit-learn tensorflow keras flask dash plotly shap
!pip install lightgbm catboost tqdm
!pip install --upgrade jupyter ipywidgets
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #

import os
import zipfile
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import torch
import time
import soft_sensor_lib
from datetime import datetime
from sklearn.linear_model import ElasticNet

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import (
    mean_absolute_percentage_error,
    mean_absolute_error,
    r2_score,
    make_scorer,
)
from lightgbm import LGBMRegressor
from tqdm.auto import tqdm
# from catboost import CatBoostRegressor
# from catboost.sklearn import CatBoostRegressor
from catboost import CatBoostRegressor as _CatBoostRegressor
from sklearn.base import BaseEstimator, RegressorMixin

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # run on GPU
print(f"\nUsing device: {device}")

Looking in indexes: https://download.pytorch.org/whl/cu121


ImportError: cannot import name '_CatBoostRegressor' from 'catboost' (/usr/local/lib/python3.12/dist-packages/catboost/__init__.py)

---
# II. Exploratory Data Analysis & Reconstruction (Quality Prediction in a Mining Process)

This section conducts Exploratory Data Analysis for the Quality Prediction in a Mining Process dataset that can be found at:
    https://www.kaggle.com/datasets/edumagalhaes/quality-prediction-in-a-mining-process

## 1. Loading Dataset, Datetime Conversion and Indexing

In [ ]:
from soft_sensor_lib import SoftSensorPipeline
# Unzip dataset only if needed
if not os.path.exists("/content/quality_prediction_in_a_mining_process.csv"):
    try:
        with zipfile.ZipFile("/content/quality_prediction_in_a_mining_process.zip", 'r') as zip_ref:
            zip_ref.extractall()
        print("Dataset extracted successfully.")
    except zipfile.BadZipFile:
        print("Error: The file 'quality_prediction_in_a_mining_process.zip' is not a valid zip file.")
        print("Please ensure you have uploaded the correct and uncorrupted zip file, or the 'quality_prediction_in_a_mining_process.csv' file directly.")
        raise # Re-raise the exception to stop execution and alert the user.
    except FileNotFoundError:
        print("Error: 'quality_prediction_in_a_mining_process.zip' not found.")
        print("Please upload the dataset to the Colab environment.")
        raise
    except Exception as e:
        print(f"An unexpected error occurred during file extraction: {e}")
        raise

# Load dataset
path = "quality_prediction_in_a_mining_process.csv"

# Datetime Conversion and Indexing
df_mining = SoftSensorPipeline.load_csv_with_date(path)

# Display Dataset Head
print('df_mining ', df_mining.shape, ':', sep='')
with pd.option_context('display.max_columns', None):
    display(df_mining.head())

## 2. Dataset Summary and Descriptive Statistics

In [ ]:
print("========= Dataset Summary =========")
print(df_mining.info())

In [ ]:
print("====== Descriptive Statistics ======")
with pd.option_context('display.max_columns', None):
    display(df_mining.describe())

## 3. Assess Completeness of every 1hr Block: Add padding for incomplete hours (rows ≠ 180)

In [ ]:
print("========= Missisng Values =========")
with pd.option_context('display.max_columns', None):
    display(df_mining.isna().sum().to_frame().T)

print("========= Index Counts =========")
idx_counts = df_mining.index.value_counts().sort_index()
display(idx_counts.to_frame().T)

print("========= Missing Indices (rows ≠ 180) =========")
missing = idx_counts[idx_counts != 180]
display(missing.to_frame().T)

df_mining = SoftSensorPipeline.pad_incomplete_hour_blocks(df_mining)

if len(df_mining.index.value_counts().loc[lambda x: x != 180]) == 0:
    print('\n✅ Missing rows have been handled correctly. New df_mining.shape:', df_mining.shape)
else:
    print('\n⚠️ Error: Dataset still has missing rows. df_mining.shape:', df_mining.shape)

## 4. Continuity Assesment (Missing hour ranges)

In [ ]:
idx_hours = df_mining.index.sort_values().unique()
expected_hours = pd.date_range(idx_hours.min(), idx_hours.max(), freq='h')
missing_hours = expected_hours.difference(idx_hours)

if not missing_hours.empty:
    s = missing_hours.to_series()
    groups = (s.diff() > pd.Timedelta(hours=1)).cumsum()
    ranges = s.groupby(groups).agg(start='min', end='max', count='size')
    print("\nMissing hour ranges:")
    display(ranges)

In [ ]:
########### Visualize Missing Hour Range ###########
silica_over_time = df_mining['% Silica Concentrate']
plt.figure(figsize=(22,4))
plt.plot(silica_over_time.index, silica_over_time)
plt.title('% Silica Concentrate Over Time')
plt.xlabel('Time')
plt.ylabel('% Silica')
plt.show()

iron_over_time = df_mining['% Iron Concentrate']
plt.figure(figsize=(22,4))
plt.plot(iron_over_time.index, iron_over_time)
plt.title('% Iron Concentrate Over Time')
plt.xlabel('Time')
plt.ylabel('% Iron')
plt.show()

## 5. Reconstruct Timestamps for Continuous 20s and 1min Resolution Dataframes (no missing hours)

In [ ]:
df_20s, df_1min = SoftSensorPipeline.reconstruct_20s_and_1min(df_mining)

# Display resampled dataframes
with pd.option_context('display.max_columns', None):
    print('df_20s ', df_20s.shape, ':', sep='')
    display(df_20s)

    print('\ndf_1min ', df_1min.shape, ':', sep='')
    display(df_1min)

## 6. Compare 20s vs 1min Descriptive Statistics

In [ ]:
with pd.option_context('display.max_columns', None):
    print("====== df_20s Statistics ======")
    display(df_20s.describe())
    print("NaN count:")
    display(df_20s.isna().sum().to_frame().T)

    print("\n====== df_1min Statistics ======")
    display(df_1min.describe())
    print("NaN count:")
    display(df_1min.isna().sum().to_frame().T)

## 7. Compare 20s vs 1min Correlations

In this section we can see that 20sec vs 1min correlation patterns are very similar

In [ ]:
# --- 20-second correlations ---
plt.figure(figsize=(18,12))
sns.heatmap(df_20s.corr(), annot=True, linewidths=0.003)
plt.title('Correlation Heatmap (20-Second Resolution)', fontsize=20)
plt.show()

In [ ]:
# --- 1-minute correlations ---
plt.figure(figsize=(18,12))
sns.heatmap(df_1min.corr(), annot=True, linewidths=0.003)
plt.title('Correlation Heatmap (1-min Resolution)', fontsize=20)
plt.show()

## 8. Target (% Silica Concentrate) Continuity Assesment
**Should see a visible gap this time because NaN has been filled in for missing timestamps**

In [ ]:
# Plot time trend
silica_over_time = df_1min['% Silica Concentrate']
plt.figure(figsize=(22,4))
plt.plot(silica_over_time.index, silica_over_time)
plt.title('% Silica Concentrate Over Time')
plt.xlabel('Time')
plt.ylabel('% Silica')
plt.show()

# Apply Rolling Statistics
rolling_window = 60*24 # one day rolling window
silica_rolling_mean = silica_over_time.rolling(window=rolling_window, center=True).mean()
silica_rolling_std = silica_over_time.rolling(window=rolling_window, center=True).std()

# Plot time trend will rolling stats
plt.figure(figsize=(22,4))
plt.plot(silica_rolling_mean.index, silica_rolling_mean, label='24-Hour Rolling Mean of % Silica')
plt.fill_between(silica_rolling_std.index, silica_rolling_mean - silica_rolling_std, silica_rolling_mean + silica_rolling_std, alpha=0.2, label='24-Hour Rolling Std of % Silica')
plt.title('% Silica Concentrate Rolliing Stats Over Time')
plt.xlabel('Time')
plt.ylabel('% Silica')
plt.legend()
plt.show()

## 9. Frequency Distributions

In [ ]:
# Select only numeric columns (ignores timestamp + categorical)
num_cols = df_mining.select_dtypes(include=[np.number]).columns

# Layout (adjust as needed)
n_cols = 4
n_rows = int(np.ceil(len(num_cols) / n_cols))

plt.figure(figsize=(20, 4 * n_rows))

for i, col in enumerate(num_cols, 1):
    plt.subplot(n_rows, n_cols, i)
    sns.histplot(df_mining[col], bins=30, kde=True)
    # plt.hist(df_mining[col], bins=50, alpha=0.7)
    plt.title(col, fontsize=10)
    plt.tight_layout()

plt.show()

---
# III. Baseline Model
This section trains and test our baseline (Linear Regression) model

## 1. Train + Test Split Helpers

In [ ]:
# Training on first 85% of time series and test on rest 15%
train_85_test_15_split = SoftSensorPipeline.train_85_test_15_split

# Training on first row of every hour block (as recommended by Professor & TA)
train_1_test_59_split = SoftSensorPipeline.train_1_test_59_split

## 2. Baseline Training & Results

In [ ]:
train_test_on_baseline_model = SoftSensorPipeline.train_test_on_baseline_model

train_test_on_baseline_model(df_1min, train_85_test_15_split, False)

---
# IV. Feature Engineering

This section covers addition of new features in order to help predictions. Both Hour-Level and Minute-Level features were added very carefully without introducing any data leaks. Date leaking is very easy with this dataset because there are reapeating targets for many rows. We also added calendar features

In [ ]:
df_all_features = SoftSensorPipeline.make_features(df_1min)

---
# V. ML Pipeline for Hyperparameter Grid Search

Elasticnet, Lightgbm and Catboost were used here because they don't reequire consistent temporal structure and are able to treat every row independently (this includes freature engineered entries in the row for temporal context)

## 1. Build models + param grids

In [ ]:
# class CatBoostRegressorSK(_CatBoostRegressor, BaseEstimator, RegressorMixin):
#     """Thin sklearn-compatible wrapper around CatBoostRegressor."""
#     pass

# ---------------------------------------------------
# Helper: build models + param grids
# ---------------------------------------------------

make_models = SoftSensorPipeline.make_models

## 2. Hyperparameter search + evaluation

In [ ]:
# ---------------------------------------------------
# Helper: run hyperparameter search + evaluation
# ---------------------------------------------------
tune_and_evaluate_models = SoftSensorPipeline.tune_and_evaluate_models

## 3. Model Comparisons Method

In [ ]:
# ---------------------------------------------------
# Orchestrator: one call
# ---------------------------------------------------
run_model_comparison = SoftSensorPipeline.run_model_comparison

## 4. Run Model Comparison (85-15 split)

In [ ]:
results1, summary1 = run_model_comparison(df_all_features, train_85_test_15_split, folds=3, n_jobs=2, debug=False)

## 5. Run Model Comparison (1-59 split as per Professor & TA recommendation)

In [ ]:
results2, summary2 = run_model_comparison(df_all_features, train_1_test_59_split, folds=3, n_jobs=2, debug=False)

## 6. Analysis of results

The 85–15 split represents true forward-in-time generalization, so the models score in a reasonable range (MAPE ~0.19–0.24, R² ~0.60–0.65). This reflects the actual difficulty of predicting silica across different months and operating regimes. When switching to the recommended 1–59 split, performance improves for all models, but LightGBM in particular shows a dramatic jump (MAPE ~0.11, R² ~0.91). This reflects how the task being evaluated is different: under the 1–59 split, the model is not forecasting into the future but interpolating within the same hour, where the target (% Silica Concentrate) is constant for all 60 minutes blocks. Because the test samples come from the same hour as a training sample, the learning problem becomes far easier and variance in the target collapses, inflating R².

So while the 1–59 split results are valid for assessing a “soft sensor” that fills in the 59 minutes between hourly lab samples, they should not be interpreted as long-term predictive accuracy. The 85–15 results better represent forecasting difficulty, whereas the 1–59 results represent interpolation accuracy. LightGBM’s very strong scores arise from the structure of the data and the easier evaluation setting, not from suspicious behavior.
